**Lab 3: Spectral Representation**

The goal of this lab is to gain familiarity with the spectral representations of signals, specially the spectrograms.

In [ ]:
import os
import numpy as np
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
from IPython.display import Audio

from util import load_audio, plot_signals, plot_spectrogram, plot_mean_spectrogram, plot_spectrum_at

First upload your reference signal and plot the first seconds of it.

In [ ]:
filepath = "celtic-harp-c5.wav"
ref, fs = load_audio(filepath)
Audio(ref, rate=fs)
T = 0.001882
f0 = 1 / T
t_start, t_end = 0.19, 0.19 + 8*T
plot_signals(ref, fs, t_start, t_end)

# **Exercises**

**1. Spectrograms**

A spectrogram is obtained by estimating the frequency content in short sections of the signal. The magnitude of the spectrum over individual sections is plotted as intensity or color on a two-dimensional plot versus frequency and time. The length of each section, or window length, determines the frequency resolution. Longer windows give good frequency resolution but fail to
track frequency changes well. Shorter windows have poor frequency resolution, but good tracking.

In Python the function `spectrogram` from the `scipy.signal` package will
compute the spectrogram. A common call to the function is defined as follows. No need to understand the meaning of each parameter at this stage. Note that we provide the `plot_spectrogram` function to plot the spectrogram.

In [ ]:
from scipy.signal import spectrogram

We can plot the spectrum (one slice of the spectrogram) of the signal at an specific time using the `plot_spectrum_at` function. For instance, we can see the spectrum of the signal at the 0.5 seconds:

In [ ]:
ff, tt, S_ref = spectrogram(ref, fs=fs, nperseg=1024, noverlap=1024/2, scaling='spectrum')
plot_spectrum_at(ff, tt, S_ref, 0.5)

1.1. Calculate and plot the spectrogram of your reference signal. Use `plt.ylim` to select the limits of the y axis in order to zoom in the frequency region of interest. For instance if you want to see the region between 0 and 4000 Hz, you can call `plt.ylim([0, 4000])` after the `plot_spectrogram` function.


In [ ]:
plot_spectrogram(ff, tt, S_ref)
plt.ylim(0,4000)
plt.title("Spectrogram of Celtic Harp C5")
plt.show()

1.2. Select a time where almost all harmonics are present and plot the spectrum at that time.

In [ ]:
time = 0.6
plot_spectrum_at(ff, tt, S_ref, time)

1.3. Use the cursor for measuring the weights of the fundamental frequency and some harmonics (6-10)

In [ ]:
weights = [1, 0.11, 0.52, 0.03, 0.00046, 0.0022 , 0.00018 , 0.0001]

**2. Synthesis**:

Let's define a function to synthetize an harmonic singal which receives the fundamental frequency ($f_0$) and the weights ($A_k$) of each harmonic and the time vector ($t$). This is similar to what you did in Lab 3- Ex3.2.


In [ ]:
def synthesize(f0, phi, Ak, t):
  y = 0
  for k in range(1, len(Ak) + 1):
    y += Ak[k-1] * np.cos(2*np.pi*k*f0*t + k*phi - (k-1)*np.pi/2)
  return y

2.1. Use the `synthetize` function to generate a synthesis with the weights ($A_k$) found in 1.3 and the fundamental frequency and phases found in previous labs. Plot both the reference and the synthetize signal. Listen to the synthetize signal.

In [ ]:
t = np.arange(0, len(ref) / fs, 1/fs) 
phi = 3*np.pi/2.1

synthesized = synthesize(f0, phi, weights, t)
plot_signals([ref, synthesized], fs, t_start, t_end)
print("\nSynthesized audio:")
Audio(synthesized, rate=fs)

2.2. Calculate the spectrogram of the synthesized signal `S_synt`; and compare the spectrums of both signals at the same time using `plot_spectrum_at(ff, tt, [S_ref, S_synt], time)`, where `S_ref`is spectrogram of the reference signal.

**Note:** use the same window length to calculate both spectrograms.

In [ ]:
ff , tt , S_synt = spectrogram(synthesized, fs=fs,nperseg=1024, noverlap=1024//2, scaling='spectrum')
plot_spectrogram(ff, tt, S_synt)
plt.ylim(0, 5000)
plt.title('Spectrogram of the Synthesized Signal')
plt.show()
plot_spectrum_at(ff, tt, [S_ref, S_synt], time)


2.3. Compare the spectrograms of the two signals. What are the main differences?

In [ ]:
plot_spectrogram(ff, tt, S_ref)
plt.ylim(0, 5000)
plt.title('Spectrogram of the Original Signal')
plt.show()

plot_spectrogram(ff, tt, S_synt)
plt.ylim(0, 5000)
plt.title('Spectrogram of the Synthesized Signal')
plt.show()

The main differences between the reference and synthesized spectrograms are in their time behavior and frequency content. The reference signal lasts about 3.5 seconds and shows clear decay over time, with higher harmonics fading before the end. It has a broader spectral bandwidth (around 737 Hz) and a mean spectral centroid of about 828 Hz, giving it a richer and more natural sound. In contrast, the synthesized signal maintains constant harmonic power throughout the same 3.5-second duration, with a narrower bandwidth (around 430 Hz) and a mean centroid of about 750 Hz. Its energy is mainly concentrated in the first few harmonics (up to about 8 × f₀ ≈ 4000 Hz). Overall, the reference sounds fuller and more complex, while the synthesized version is clearer but simpler and more tonal.

2.4. Listen to the two audios (reference and synthesized). What are the main differences?

In [ ]:
print("Reference audio:")
display(Audio(ref, rate=fs))

print("\nSynthesized audio:")
display(Audio(synthesized, rate=fs))

The main differences between the reference and synthesized audios are in their loudness change and sound quality. The reference audio naturally gets quieter over its 3.5 seconds, with higher harmonics fading over time, making it sound warmer and more realistic. The synthesized audio, made of perfect sine waves, keeps the same loudness and has fewer harmonics, so it sounds cleaner but more artificial.